In [1]:
import pandas as pd

In [2]:
import math

def utmToLatLng(zone, easting, northing, northernHemisphere=True):
    if not northernHemisphere:
        northing = 10000000 - northing

    a = 6378137
    e = 0.081819191
    e1sq = 0.006739497
    k0 = 0.9996

    arc = northing / k0
    mu = arc / (a * (1 - math.pow(e, 2) / 4.0 - 3 * math.pow(e, 4) / 64.0 - 5 * math.pow(e, 6) / 256.0))

    ei = (1 - math.pow((1 - e * e), (1 / 2.0))) / (1 + math.pow((1 - e * e), (1 / 2.0)))

    ca = 3 * ei / 2 - 27 * math.pow(ei, 3) / 32.0

    cb = 21 * math.pow(ei, 2) / 16 - 55 * math.pow(ei, 4) / 32
    cc = 151 * math.pow(ei, 3) / 96
    cd = 1097 * math.pow(ei, 4) / 512
    phi1 = mu + ca * math.sin(2 * mu) + cb * math.sin(4 * mu) + cc * math.sin(6 * mu) + cd * math.sin(8 * mu)

    n0 = a / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (1 / 2.0))

    r0 = a * (1 - e * e) / math.pow((1 - math.pow((e * math.sin(phi1)), 2)), (3 / 2.0))
    fact1 = n0 * math.tan(phi1) / r0

    _a1 = 500000 - easting
    dd0 = _a1 / (n0 * k0)
    fact2 = dd0 * dd0 / 2

    t0 = math.pow(math.tan(phi1), 2)
    Q0 = e1sq * math.pow(math.cos(phi1), 2)
    fact3 = (5 + 3 * t0 + 10 * Q0 - 4 * Q0 * Q0 - 9 * e1sq) * math.pow(dd0, 4) / 24

    fact4 = (61 + 90 * t0 + 298 * Q0 + 45 * t0 * t0 - 252 * e1sq - 3 * Q0 * Q0) * math.pow(dd0, 6) / 720

    lof1 = _a1 / (n0 * k0)
    lof2 = (1 + 2 * t0 + Q0) * math.pow(dd0, 3) / 6.0
    lof3 = (5 - 2 * Q0 + 28 * t0 - 3 * math.pow(Q0, 2) + 8 * e1sq + 24 * math.pow(t0, 2)) * math.pow(dd0, 5) / 120
    _a2 = (lof1 - lof2 + lof3) / math.cos(phi1)
    _a3 = _a2 * 180 / math.pi

    latitude = 180 * (phi1 - fact1 * (fact2 + fact3 + fact4)) / math.pi

    if not northernHemisphere:
        latitude = -latitude

    longitude = ((zone > 0) and (6 * zone - 183.0) or 3.0) - _a3

    return (latitude, longitude)

In [3]:
df = pd.read_csv("./67-Insect-Populations-via-Sticky-Traps.csv",low_memory=False)

In [4]:
df = df[(df.Year>=2007)&(df.Family=='Coccinellidae')&(df.Adults!='0')].drop_duplicates(keep='first').reset_index(drop=True)

In [6]:
gps = []
for i,j in zip(df.utm_easting,df.utm_northing):
      gps.append(utmToLatLng(16,float(i),float(j)))

In [11]:
df['gps']=gps

In [12]:
df

,Sample_Date,Treatment,Replicate,Station,Species,Family,Order,Adults,utm_easting,utm_northing,Year,gps
0,2019-08-29,T1,1.0,3.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,633942.990536,4696849.015648,2019.0,"(42.41238752524203, -85.37214230398072)"
1,2019-08-29,T1,1.0,5.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,633972.990536,4696859.015648,2019.0,"(42.41247236605469, -85.37177551695578)"
2,2019-08-29,T1,3.0,2.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,634142.121438,4696162.129521,2019.0,"(42.406169499900635, -85.36988330541355)"
3,2019-08-29,T1,4.0,1.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,634210.093509,4696635.195083,2019.0,"(42.410416449112965, -85.36894726807616)"
4,2019-08-29,T1,4.0,3.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,634240.093509,4696633.195083,2019.0,"(42.41039325149842, -85.36858328887467)"
...,...,...,...,...,...,...,...,...,...,...,...,...
17918,2007-05-10,T1,4.0,5.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,634270.093509,4696643.195083,2007.0,"(42.41047808102667, -85.36821650911362)"
17919,2007-05-10,T1,5.0,1.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,634006.584312,4696968.074382,2007.0,"(42.41344834777488, -85.37134199103396)"
17920,2007-05-10,T1,5.0,3.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,634036.584312,4696966.074382,2007.0,"(42.41342515748901, -85.37097799320969)"
17921,2007-05-10,T1,5.0,5.0,Coleomegilla maculata,Coccinellidae,Coleoptera,1,634066.584312,4696976.074382,2007.0,"(42.41350999441751, -85.37061119863826)"


In [45]:
df = df[['Station', 'Species', 'Family', 'Order', 'Adults', 'Year', 'gps']]

In [46]:
df = df[['Station', 'Species', 'Family', 'Order', 'Adults', 'Year', 'gps']]

,Station,Species,Family,Order,Adults,Year,gps
0,3.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,2019.0,"(42.41238752524203, -85.37214230398072)"
1,5.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,2019.0,"(42.41247236605469, -85.37177551695578)"
2,2.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,2019.0,"(42.406169499900635, -85.36988330541355)"
3,1.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,2019.0,"(42.410416449112965, -85.36894726807616)"
4,3.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,2019.0,"(42.41039325149842, -85.36858328887467)"
...,...,...,...,...,...,...,...
17918,5.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,2007.0,"(42.41047808102667, -85.36821650911362)"
17919,1.0,Coccinella septempunctata,Coccinellidae,Coleoptera,1,2007.0,"(42.41344834777488, -85.37134199103396)"
17920,3.0,Harmonia axyridis,Coccinellidae,Coleoptera,1,2007.0,"(42.41342515748901, -85.37097799320969)"
17921,5.0,Coleomegilla maculata,Coccinellidae,Coleoptera,1,2007.0,"(42.41350999441751, -85.37061119863826)"


In [54]:
df = df[(df.Species!='Harmonia axyridis')&(df.Species!='Coccinella septempunctata')&(df.Species!='Adalia bipunctata')].reset_index(drop=True)

In [56]:
df.drop_duplicates(['gps'],keep='first')

,Station,Species,Family,Order,Adults,Year,gps
0,3.0,Hippodamia variegata,Coccinellidae,Coleoptera,1,2019.0,"(42.40729501722236, -85.3768486059242)"
1,1.0,Coleomegilla maculata,Coccinellidae,Coleoptera,1,2019.0,"(42.413470544444635, -85.36898595233609)"
2,2.0,Hippodamia variegata,Coccinellidae,Coleoptera,1,2019.0,"(42.40725043042798, -85.37005015465107)"
3,3.0,Coleomegilla maculata,Coccinellidae,Coleoptera,1,2019.0,"(42.406258415962256, -85.37203892949609)"
4,3.0,Propylea quatuorodecipuntata,Coccinellidae,Coleoptera,1,2019.0,"(42.4093467262504, -85.36975768504492)"
...,...,...,...,...,...,...,...
4144,5.0,Propylea quatuorodecipuntata,Coccinellidae,Coleoptera,1,2012.0,"(42.408404159184016, -85.39674377204561)"
4275,2.0,Psyllobora vigintimaculata,Coccinellidae,Coleoptera,1,2012.0,"(42.41456816865097, -85.37420607773437)"
4370,3.0,Psyllobora vigintimaculata,Coccinellidae,Coleoptera,1,2012.0,"(42.41236990308213, -85.38515833601457)"
4371,5.0,Psyllobora vigintimaculata,Coccinellidae,Coleoptera,1,2012.0,"(42.4123154247044, -85.38495446873031)"


In [59]:
df = df[['Year', 'gps']]

In [63]:
df.drop_duplicates(['gps'],keep='first').reset_index(drop=True).to_csv("./kellog_location.csv")